In [41]:
import os
import json
import psycopg2
from psycopg2 import sql

In [51]:
# Configurações de conexão
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)


In [52]:

# Caminho da pasta base
base_path = "D:/UFRJ/data-mining/gps"

# Estrutura SQL para inserção na tabela load
insert_load_sql = """
INSERT INTO vehicle_tracking_load (ordem, latitude, longitude, datahora, velocidade, linha, datahoraenvio, datahoraservidor)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Query de Insert com Conversão de Tipos
insert_vehicle_tracking_sql = """
INSERT INTO vehicle_tracking (
    ordem,
    latitude,
    longitude,
    datahora_epoch,
    datahora,
    velocidade,
    linha,
    datahoraenvio_epoch,
    datahoraservidor_epoch,
    datahoraservidor,
    datahoraenvio
)
SELECT
    ordem,
    latitude::DOUBLE PRECISION,
    longitude::DOUBLE PRECISION,
    datahora::BIGINT / 1000 AS datahora_epoch,
    TO_TIMESTAMP(datahora::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE AS datahora,
    velocidade::INT,
    linha,
    datahoraenvio::BIGINT / 1000 AS datahoraenvio_epoch,
    datahoraservidor::BIGINT / 1000 AS datahoraservidor_epoch,
    TO_TIMESTAMP(datahoraservidor::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE AS datahoraservidor,
    TO_TIMESTAMP(datahoraenvio::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE AS datahoraenvio
FROM vehicle_tracking_load
where TO_TIMESTAMP(datahora::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE >= '20240424_0000'
and TO_TIMESTAMP(datahora::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE < '20240512_0000';
"""


In [53]:
# Função para truncar a tabela load
def truncate_load_table(cur):
    cur.execute("TRUNCATE TABLE vehicle_tracking_load")
    conn.commit()

# Função para processar cada arquivo JSON
def process_json_file(file_path, cur):
    with open(file_path, 'r') as file:
        data_list = json.load(file)
        print(data_list[:10])
        for data in data_list:
            cur.execute(insert_load_sql, (
                data['ordem'],
                data['latitude'].replace(',', '.'),
                data['longitude'].replace(',', '.'),
                data['datahora'],
                data['velocidade'],
                data['linha'],
                data['datahoraenvio'],
                data['datahoraservidor']
            ))
    conn.commit()

    # Inserir dados na tabela final
    cur.execute(insert_vehicle_tracking_sql)
    conn.commit()

    # Truncar a tabela load para o próximo arquivo
    truncate_load_table(cur)



In [54]:
# Percorrer todas as subpastas e arquivos JSON
processed_files = ['2024-04-25_00.json',
 '2024-04-25_01.json',
 '2024-04-25_02.json',
 '2024-04-25_03.json',
 '2024-04-25_04.json',
 '2024-04-25_05.json',
 '2024-04-25_06.json',
 '2024-04-25_07.json',
 '2024-04-25_08.json',
 '2024-04-25_09.json',
 '2024-04-25_10.json',
 '2024-04-25_11.json',
 '2024-04-25_12.json',
 '2024-04-25_13.json',
 '2024-04-25_14.json',
 '2024-04-25_15.json',
 '2024-04-25_16.json',
 '2024-04-25_17.json',
 '2024-04-25_18.json',
 '2024-04-25_19.json',
 '2024-04-25_20.json',
 '2024-04-25_21.json',
 '2024-04-25_22.json',
 '2024-04-25_23.json',
 '2024-04-26_00.json',
 '2024-04-26_01.json',
 '2024-04-26_02.json',
 '2024-04-26_03.json',
 '2024-04-26_04.json',
 '2024-04-26_05.json',
 '2024-04-26_06.json',
 '2024-04-26_07.json',
 '2024-04-26_08.json',
 '2024-04-26_09.json',
 '2024-04-26_10.json',
 '2024-04-26_11.json',
 '2024-04-26_12.json',
 '2024-04-26_13.json',
 '2024-04-26_14.json',
 '2024-04-26_15.json',
 '2024-04-26_16.json',
 '2024-04-26_17.json',
 '2024-04-26_18.json',
 '2024-04-26_19.json',
 '2024-04-26_20.json',
 '2024-04-26_21.json',
 '2024-04-26_22.json',
 '2024-04-26_23.json',
 '2024-04-27_00.json',
 '2024-04-27_01.json',
 '2024-04-27_02.json',
 '2024-04-27_03.json',
 '2024-04-27_04.json',
 '2024-04-27_05.json',
 '2024-04-27_06.json',
 '2024-04-27_07.json',
 '2024-04-27_08.json',
 '2024-04-27_09.json',
 '2024-04-27_10.json',
 '2024-04-27_11.json',
 '2024-04-27_12.json',
 '2024-04-27_13.json',
 '2024-04-27_14.json',
 '2024-04-27_15.json',
 '2024-04-27_16.json',
 '2024-04-27_17.json',
 '2024-04-27_18.json',
 '2024-04-27_19.json',
 '2024-04-27_20.json',
 '2024-04-27_21.json',
 '2024-04-27_22.json',
 '2024-04-27_23.json',
 '2024-04-28_00.json',
 '2024-04-28_01.json',
 '2024-04-28_02.json',
 '2024-04-28_03.json',
 '2024-04-28_04.json',
 '2024-04-28_05.json',
 '2024-04-28_06.json',
 '2024-04-28_07.json',
 '2024-04-28_08.json',
 '2024-04-28_09.json',
 '2024-04-28_10.json',
 '2024-04-28_11.json',
 '2024-04-28_12.json',
 '2024-04-28_13.json',
 '2024-04-28_14.json',
 '2024-04-28_15.json',
 '2024-04-28_16.json',
 '2024-04-28_17.json',
 '2024-04-28_18.json',
 '2024-04-28_19.json',
 '2024-04-28_20.json',
 '2024-04-28_21.json',
 '2024-04-28_22.json',
 '2024-04-28_23.json',
 '2024-04-29_00.json',
 '2024-04-29_01.json',
 '2024-04-29_02.json',
 '2024-04-29_03.json',
 '2024-04-29_04.json',
 '2024-04-29_05.json',
 '2024-04-29_06.json',
 '2024-04-29_07.json',
 '2024-04-29_08.json',
 '2024-04-29_09.json',
 '2024-04-29_10.json',
 '2024-04-29_11.json',
 '2024-04-29_12.json',
 '2024-04-29_13.json',
 '2024-04-29_14.json',
 '2024-04-29_15.json',
 '2024-04-29_16.json',
 '2024-04-29_17.json',
 '2024-04-29_18.json',
 '2024-04-29_19.json',
 '2024-04-29_20.json',
 '2024-04-29_21.json',
 '2024-04-29_22.json',
 '2024-04-29_23.json',
 '2024-04-30_00.json',
 '2024-04-30_01.json',
 '2024-04-30_02.json',
 '2024-04-30_03.json',
 '2024-04-30_04.json',
 '2024-04-30_05.json',
 '2024-04-30_06.json',
 '2024-04-30_07.json',
 '2024-04-30_08.json',
 '2024-04-30_09.json',
 '2024-04-30_10.json',
 '2024-04-30_11.json',
 '2024-04-30_12.json',
 '2024-04-30_13.json',
 '2024-04-30_14.json',
 '2024-04-30_15.json',
 '2024-04-30_16.json',
 '2024-04-30_17.json',
 '2024-04-30_18.json',
 '2024-04-30_19.json',
 '2024-04-30_20.json',
 '2024-04-30_21.json',
 '2024-04-30_22.json',
 '2024-04-30_23.json',
 '2024-05-01_00.json',
 '2024-05-01_01.json',
 '2024-05-01_02.json',
 '2024-05-01_03.json',
 '2024-05-01_04.json',
 '2024-05-01_05.json',
 '2024-05-01_06.json',
 '2024-05-01_07.json',
 '2024-05-01_08.json',
 '2024-05-01_09.json',
 '2024-05-01_10.json',
 '2024-05-01_11.json',
 '2024-05-01_12.json',
 '2024-05-01_13.json',
 '2024-05-01_14.json',
 '2024-05-01_15.json',
 '2024-05-01_16.json',
 '2024-05-01_17.json',
 '2024-05-01_18.json',
 '2024-05-01_19.json',
 '2024-05-01_20.json',
 '2024-05-01_21.json',
 '2024-05-01_22.json',
 '2024-05-01_23.json',
 '2024-05-02_00.json',
 '2024-05-02_01.json',
 '2024-05-02_02.json',
 '2024-05-02_03.json',
 '2024-05-02_04.json',
 '2024-05-02_05.json',
 '2024-05-02_06.json',
 '2024-05-02_07.json',
 '2024-05-02_08.json',
 '2024-05-02_09.json',
 '2024-05-02_10.json',
 '2024-05-02_11.json',
 '2024-05-02_12.json',
 '2024-05-02_13.json',
 '2024-05-02_14.json',
 '2024-05-02_15.json',
 '2024-05-02_16.json',
 '2024-05-02_17.json',
 '2024-05-02_18.json',
 '2024-05-02_19.json',
 '2024-05-02_20.json',
 '2024-05-02_21.json',
 '2024-05-02_22.json',
 '2024-05-02_23.json',
 '2024-05-03_00.json',
 '2024-05-03_01.json',
 '2024-05-03_02.json',
 '2024-05-03_03.json',
 '2024-05-03_04.json',
 '2024-05-03_05.json',
 '2024-05-03_06.json',
 '2024-05-03_07.json',
 '2024-05-03_08.json',
 '2024-05-03_09.json',
 '2024-05-03_10.json',
 '2024-05-03_11.json',
 '2024-05-03_12.json',
 '2024-05-03_13.json',
 '2024-05-03_14.json',
 '2024-05-03_15.json',
 '2024-05-03_16.json',
 '2024-05-03_17.json',
 '2024-05-03_18.json',
 '2024-05-03_19.json',
 '2024-05-03_20.json',
 '2024-05-03_21.json',
 '2024-05-03_22.json',
 '2024-05-03_23.json',
 '2024-05-04_00.json',
 '2024-05-04_01.json',
 '2024-05-04_02.json',
 '2024-05-04_03.json',
 '2024-05-04_04.json',
 '2024-05-04_05.json',
 '2024-05-04_06.json', 
 '2024-05-04_07.json']

cur = conn.cursor()
for root, dirs, files in os.walk(base_path):
    for file in files:
        print('Processing', file)
        if file.endswith(".json") and file not in processed_files:
            file_path = os.path.join(root, file)
            processed_files.append(file)
            process_json_file(file_path, cur)
            

# Fechar a conexão
cur.close()
conn.close()

Processing 2024-04-25_00.json
Processing 2024-04-25_01.json
Processing 2024-04-25_02.json
Processing 2024-04-25_03.json
Processing 2024-04-25_04.json
Processing 2024-04-25_05.json
Processing 2024-04-25_06.json
Processing 2024-04-25_07.json
Processing 2024-04-25_08.json
Processing 2024-04-25_09.json
Processing 2024-04-25_10.json
Processing 2024-04-25_11.json
Processing 2024-04-25_12.json
Processing 2024-04-25_13.json
Processing 2024-04-25_14.json
Processing 2024-04-25_15.json
Processing 2024-04-25_16.json
Processing 2024-04-25_17.json
Processing 2024-04-25_18.json
Processing 2024-04-25_19.json
Processing 2024-04-25_20.json
Processing 2024-04-25_21.json
Processing 2024-04-25_22.json
Processing 2024-04-25_23.json
Processing 2024-04-26_00.json
Processing 2024-04-26_01.json
Processing 2024-04-26_02.json
Processing 2024-04-26_03.json
Processing 2024-04-26_04.json
Processing 2024-04-26_05.json
Processing 2024-04-26_06.json
Processing 2024-04-26_07.json
Processing 2024-04-26_08.json
Processing



alter table vehicle_tracking add column geom GEOGRAPHY(Point, 4326)


update vehicle_tracking
set geom = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326)::geography